In [1]:
% matplotlib inline
from pymongo import MongoClient
import pymongo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import seaborn as sns
import networkx as nx
import nltk
from langdetect import detect
from langdetect import detect_langs
from langdetect import DetectorFactory
from nltk.tokenize import RegexpTokenizer  
from stop_words import get_stop_words  
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from gensim import corpora, models  
import gensim  


C:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
mongo_username = 'Campione'
mongo_password = 'veTRxJL29lpKWwPn'
mongo_url = 'mongodb://%s:%s@cluster0-shard-00-01-i6gcp.mongodb.net:27017/admin' % (
            mongo_username, mongo_password)
client = MongoClient(mongo_url,ssl=True,replicaSet='Cluster0-shard-0',authSource='admin')
info = client.server_info()  # Forces a call.
# print(info)
print(client.database_names())

['gitdb', 'gitdbPro', 'guidebook', 'test', 'admin', 'local']


In [3]:
db = client.gitdbPro
repos = db.repos
descriptions = repos.distinct('description')
df = pd.DataFrame(descriptions)
df.head()

,0
0,How to be low-level programmer
1,Import OpenStreetMap data into Unreal Engine 4
2,react-native template to target multiple platf...
3,Connect your App to Multiple Messaging Channel...
4,Proto Actor - Ultra fast distributed actors fo...


In [4]:
df[0]

0                           How to be low-level programmer
1           Import OpenStreetMap data into Unreal Engine 4
2        react-native template to target multiple platf...
3        Connect your App to Multiple Messaging Channel...
4        Proto Actor - Ultra fast distributed actors fo...
5                用rxjava简单封装了指纹识别，并实现对生命周期的自动绑定，顺便撸了一个指纹控件
6                                   pagehelper-spring-boot
7            📅 A date picker components for ionic2 ／ionic3
8        Track prices on Amazon and receive email alert...
9        Advanced web based trading client for the Stel...
10                                  Join our slack channel
11       An Elm-based prototype to help designers build...
12       Can neural networks transliterate Romaji into ...
13                              A repo with small projects
14               Go - CQRS / Event Sourcing made easy - Go
15                                RNN Tutorial for Artists
16       A macOS app to convert JSON objects into Swift.

In [5]:
DetectorFactory.seed = 0

In [6]:
cn_text_list = []
en_text_list = []
for item in df[0]:
    try:
        if detect(str(item)) == 'zh-cn':
            cn_text_list.append(item)
        else:
            en_text_list.append(item)
    except Exception as e:
        print(str(e))

No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.


In [7]:
len(cn_text_list)

697

In [8]:
len(en_text_list)

11143

In [9]:
df_cn = pd.DataFrame({'text':cn_text_list})
print(df_cn.head())
df_cn.to_csv('cn_text')

                                                text
0          用rxjava简单封装了指纹识别，并实现对生命周期的自动绑定，顺便撸了一个指纹控件
1           每周一篇，内容精简，不咸不淡，期盼探讨。微信公众号：芋道源码【纯源码分享公众号】
2                                  iOS即时通讯，从入门到“放弃”？
3  『假装看天气』─── 天气预报 & 公交查询 & 资讯阅读 & 妹纸福利 的 Android...
4                                         微信小程序－日历 📅


In [10]:
df_en = pd.DataFrame({'text':en_text_list})
print(df_en.head())
df_en.to_csv('en_text')

                                                text
0                     How to be low-level programmer
1     Import OpenStreetMap data into Unreal Engine 4
2  react-native template to target multiple platf...
3  Connect your App to Multiple Messaging Channel...
4  Proto Actor - Ultra fast distributed actors fo...


In [19]:
# GENERATE LDA Model
import re

# list for tokenized documents in loop  
texts = []
tokenizer = RegexpTokenizer(r'\w+')
# create English stop words list
en_stop = get_stop_words('en')
# Create p_stemmer of class PorterStemmer
stemmer = SnowballStemmer("english")

# loop through document list  
for i in en_text_list:
    if i is not None:
        raw = i.lower()
        # clean and tokenize document string
        tokens = tokenizer.tokenize(raw)
        
        # remove stop words from tokens  
        stopped_tokens = [i for i in tokens if not i in en_stop]
        
        # remove all numbers
        without_numbers = [re.sub(r'\d+', '', i) for i in stopped_tokens]
        
        # stem tokens
        stemmed_tokens = [stemmer.stem(i) for i in without_numbers]
        
        # add tokens to list
        texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
      
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=20, id2word = dictionary, passes=20)

In [21]:
for topic in ldamodel.print_topics(num_words=5, num_topics=20):
    print(topic)

(0, '0.082*"implement" + 0.056*"learn" + 0.046*"network" + 0.038*"deep" + 0.032*"pytorch"')
(1, '0.059*"sourc" + 0.049*"open" + 0.017*"plugin" + 0.014*"webpack" + 0.014*"work"')
(2, '0.033*"test" + 0.022*"kubernet" + 0.018*"docker" + 0.014*"deploy" + 0.014*"tool"')
(3, '0.069*"generat" + 0.023*"text" + 0.019*"maco" + 0.019*"org" + 0.013*"code"')
(4, '0.021*"googl" + 0.020*"client" + 0.020*"progress" + 0.015*"blockchain" + 0.014*"show"')
(5, '0.031*"extens" + 0.025*"browser" + 0.020*"json" + 0.018*"repositori" + 0.017*"chrome"')
(6, '0.027*"time" + 0.025*"code" + 0.022*"go" + 0.021*"api" + 0.019*"develop"')
(7, '0.119*"react" + 0.052*"nativ" + 0.051*"app" + 0.043*"compon" + 0.021*"render"')
(8, '0.073*"list" + 0.044*"awesom" + 0.037*"resourc" + 0.035*"curat" + 0.020*"relat"')
(9, '0.025*"java" + 0.019*"model" + 0.019*"d" + 0.019*"train" + 0.018*"cryptocurr"')
(10, '0.058*"vue" + 0.042*"js" + 0.019*"user" + 0.016*"s" + 0.014*"aw"')
(11, '0.034*"python" + 0.028*"learn" + 0.025*"program" +

In [23]:
from gensim.models.wrappers import LdaMallet

mallet_path = r"/mallet-2.0.8/bin/mallet"
model = LdaMallet(mallet_path, corpus, num_topics=10, id2word=dictionary)
print(model[corpus])

CalledProcessError: Command '/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input C:\Users\WENXUA~1\AppData\Local\Temp\a1a921_corpus.txt --output C:\Users\WENXUA~1\AppData\Local\Temp\a1a921_corpus.mallet' returned non-zero exit status 1.

In [40]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
12     7.541676        1       1  0.198403  0.071727
14     6.324494        1       2 -0.163561  0.142086
18     6.313152        1       3  0.132803  0.080098
7      6.098359        1       4  0.065522 -0.072171
1      5.845858        1       5  0.102236 -0.034642
13     5.709390        1       6 -0.083050 -0.163709
17     5.352720        1       7  0.057477 -0.119346
5      5.326530        1       8  0.081110  0.050464
9      5.126689        1       9  0.032001 -0.082680
8      4.896264        1      10  0.072945 -0.067656
10     4.790162        1      11 -0.179939 -0.143373
16     4.617206        1      12 -0.074602 -0.059766
0      4.553503        1      13 -0.010797 -0.043646
11     4.500832        1      14  0.030833 -0.045222
15     4.137219        1      15  0.037259  0.060682
4      4.117423        1      16  0.061827  0.103666
2      3.970478        1      17 -0.058143  0.018014
3      3.712804        1      18 -0.045915  0.094722
19     3.537668        1      19 -0.140432  0.195140
6      3.527573        1      20 -0.115977  0.015611, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
16    Default  492.000000          react  492.000000  30.0000  30.0000
115   Default  417.000000          learn  417.000000  29.0000  29.0000
101   Default  441.000000      implement  441.000000  28.0000  28.0000
54    Default  280.000000        network  280.000000  27.0000  27.0000
180   Default  266.000000            vue  266.000000  26.0000  26.0000
39    Default  317.000000         compon  317.000000  25.0000  25.0000
81    Default  231.000000          sourc  231.000000  24.0000  24.0000
92    Default  208.000000          swift  208.000000  23.0000  23.0000
31    Default  242.000000             go  242.000000  22.0000  22.0000
178   Default  308.000000             js  308.000000  21.0000  21.0000
495   Default  225.000000           list  225.000000  20.0000  20.0000
233   Default  287.000000           http  287.000000  19.0000  19.0000
14    Default  218.000000          nativ  218.000000  18.0000  18.0000
237   Default  283.000000         python  283.000000  17.0000  17.0000
24    Default  189.000000           open  189.000000  16.0000  16.0000
1293  Default  194.000000        pytorch  194.000000  15.0000  15.0000
144   Default  432.000000        android  432.000000  14.0000  14.0000
153   Default  254.000000           imag  254.000000  13.0000  13.0000
114   Default  197.000000           deep  197.000000  12.0000  12.0000
116   Default  158.000000         machin  158.000000  11.0000  11.0000
188   Default  147.000000           time  147.000000  10.0000  10.0000
219   Default  193.000000         server  193.000000   9.0000   9.0000
440   Default  139.000000           line  139.000000   8.0000   8.0000
28    Default  191.000000              c  191.000000   7.0000   7.0000
15    Default  150.000000       platform  150.000000   6.0000   6.0000
20    Default  428.000000            app  428.000000   5.0000   5.0000
641   Default  161.000000     tensorflow  161.000000   4.0000   4.0000
207   Default  190.000000     javascript  190.000000   3.0000   3.0000
477   Default  157.000000           view  157.000000   2.0000   2.0000
437   Default  129.000000        command  129.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
2258  Topic20   17.962132          arxiv   18.787497   3.2996  -5.1060
2453  Topic20   17.691443          latex   18.516808   3.2990  -5.1212
3020  Topic20   17.018528     comprehens   17.843892   3.2972  -5.1600
6     Topic20   16.847056         import   17.672420   3.2967  -5.1701
3541  Topic20   16.265270          index   17.090634   3.2951  -5.2053
2257  Topic20   14.674304             ab   15.499668   3.2898  -5.3082
1649  Topic20   14.553

In [43]:
# SKLEARN LEARN LDA
from __future__ import print_function
from time import time
from random import shuffle

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

n_samples = 7000
n_features = 100
n_components = 5
n_top_words = 20

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [18]:
# shuffle the data
en_shuffle = en_text_list
shuffle(en_shuffle)
print(en_shuffle)
len(en_shuffle)

['A set of tutorials about code injection for Windows.', 'A Django template for using Flask for the frontend, Django for the backend. ', 'Digger is a lightweight download framework that requires only one line of code to complete the file download task', 'Handwritten digits classification from MNIST with TensorFlow on Android; Featuring Tutorial!', 'Lockscreen script depended on i3lock', 'A collection of scripts which may come in handy during your freedom fighting activities.', 'Awesome list of `now.sh` deployments. ', 'A web based terminal for EC2 instances that does not require SSH or any other inbound connections to the instaces. Instead it uses the AWS Systems Manager (SSM) API to run commands (bash or powershell).', 'Colorify - simple, yet powerful color library.', 'Web framework for the zen of python.', 'Spring Security Tutorial takes you to learn Spring Security step by step with a large number of samples. Spring Security Tutorial 是一本关于 Spring Security 学习的开源书。利用业余时间写了本书，图文并茂，用大量实

11143

In [19]:
# Get data sample
en_samples = en_shuffle[:n_samples]
len(en_samples)

7000

In [20]:
# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(en_samples)
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
dtm_tf = tf_vectorizer.fit_transform(en_samples)
# tfid_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(en_samples)
print("done in %0.3fs." % (time() - t0))

Extracting tf-idf features for NMF...
done in 0.112s.
Extracting tf features for LDA...
done in 0.198s.


In [21]:
print(dtm_tf.shape)
print(dtm_tf)
print(dtm_tfidf.shape)
print(dtm_tfidf)

(7000, 100)
  (0, 98)	1
  (0, 14)	1
  (1, 93)	1
  (2, 31)	1
  (2, 51)	1
  (2, 32)	1
  (2, 49)	1
  (2, 14)	1
  (3, 1)	1
  (3, 85)	1
  (5, 15)	1
  (6, 53)	1
  (6, 8)	1
  (7, 2)	1
  (7, 9)	1
  (7, 97)	1
  (8, 48)	1
  (8, 81)	1
  (9, 73)	1
  (9, 97)	1
  (9, 32)	1
  (10, 79)	5
  (11, 37)	1
  (11, 14)	1
  (12, 47)	1
  :	:
  (6991, 0)	1
  (6991, 73)	1
  (6992, 78)	1
  (6993, 28)	1
  (6993, 25)	1
  (6993, 37)	1
  (6993, 98)	1
  (6994, 68)	1
  (6995, 80)	1
  (6995, 6)	1
  (6995, 43)	1
  (6995, 12)	1
  (6995, 75)	2
  (6995, 50)	1
  (6995, 32)	1
  (6996, 23)	1
  (6996, 5)	1
  (6997, 87)	1
  (6997, 23)	1
  (6998, 9)	1
  (6998, 32)	1
  (6998, 14)	1
  (6999, 21)	1
  (6999, 51)	1
  (6999, 32)	1
(7000, 100)
  (0, 14)	0.592660790246
  (0, 98)	0.805452163512
  (1, 93)	1.0
  (2, 14)	0.365388811836
  (2, 49)	0.503065289492
  (2, 32)	0.388036436956
  (2, 51)	0.472606439903
  (2, 31)	0.489374301788
  (3, 85)	0.772092977534
  (3, 1)	0.635509586113
  (5, 15)	1.0
  (6, 8)	0.731288788942
  (6, 53)	0.68206796374

In [22]:
# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(dtm_tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(lda, tfidf_feature_names, n_top_words)

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=7000 and n_features=100...
done in 0.190s.

Topics in NMF model (Frobenius norm):
Topic #0: library javascript data python lightweight fast source easy text image swift java like open api rust performance create platform file
Topic #1: code source learning deep open 2017 https project machine text networks rust com model github javascript tool tensorflow plugin swift
Topic #2: react native app components component ios web built create apps ui like build design js use development support application management
Topic #3: android app ios view kotlin architecture project use apps easy framework client native way java plugin ui file github application
Topic #4: simple using based js implementation vue python framework tool learning pytorch web data deep api tensorflow node use project machine

Fitting the NMF model (generalized Kullback-Leibler divergence) with tf-idf features, n_samples=7000 and n_features=100...
done i

In [23]:
# print(lda.components_)
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
pyLDAvis.sklearn.prepare(lda, dtm_tfidf, tf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      23.276987        1       1 -0.120829  0.370829
3      21.823354        1       2 -0.117974 -0.366783
2      19.188181        1       3 -0.415275 -0.010125
4      18.194534        1       4  0.318163 -0.244459
1      17.516943        1       5  0.335915  0.250538, topic_info=     Category        Freq            Term       Total  loglift  logprob
term                                                                   
1     Default  193.000000         android  193.000000  30.0000  30.0000
47    Default  166.000000        learning  166.000000  29.0000  29.0000
75    Default  187.000000           react  187.000000  28.0000  28.0000
48    Default  241.000000         library  241.000000  27.0000  27.0000
40    Default  159.000000             ios  159.000000  26.0000  26.0000
88    Default  147.000000            tool  147.000000  25.0000  25.0000
81    Default  163.000000          simple  163.000000  24.0000  24.0000
73    Default  137.000000          python  137.000000  23.0000  23.0000
9     Default  147.000000           based  147.000000  22.0000  22.0000
39    Default  129.000000  implementation  129.000000  21.0000  21.0000
3     Default  137.000000             app  137.000000  20.0000  20.0000
82    Default  110.000000          source  110.000000  19.0000  19.0000
43    Default  127.000000              js  127.000000  18.0000  18.0000
93    Default  173.000000           using  173.000000  17.0000  17.0000
84    Default  108.000000           swift  108.000000  16.0000  16.0000
74    Default   97.000000         pytorch   97.000000  15.0000  15.0000
24    Default   96.000000            deep   96.000000  14.0000  14.0000
14    Default  178.000000            code  178.000000  13.0000  13.0000
97    Default  114.000000             web  114.000000  12.0000  12.0000
95    Default  111.000000             vue  111.000000  11.0000  11.0000
65    Default   93.000000            open   93.000000  10.0000  10.0000
23    Default  102.000000            data  102.000000   9.0000   9.0000
4     Default   85.000000     application   85.000000   8.0000   8.0000
53    Default   86.000000            list   86.000000   7.0000   7.0000
80    Default   83.000000          server   83.000000   6.0000   6.0000
2     Default   92.000000             api   92.000000   5.0000   5.0000
58    Default   85.000000          native   85.000000   4.0000   4.0000
38    Default   77.000000           image   77.000000   3.0000   3.0000
50    Default   72.000000            like   72.000000   2.0000   2.0000
42    Default   81.000000      javascript   81.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
98     Topic5   50.906090         windows   51.395539   1.7324  -3.2597
59     Topic5   48.624953             net   49.095464   1.7324  -3.3055
0      Topic5   58.955836            2017   59.596169   1.7312  -3.1129
55     Topic5   43.198459            make   43.683177   1.7308  -3.4239
94     Topic5   64.047819            view   65.005929   1.7272  -3.0301
62     Topic5   53.833401          neural   54.764900   1.7248  -3.2038
61     Topic5   54.210211        networks   56.559707   1.6996  -3.1968
11     Topic5   34.365033           build   50.777313   1.3516  -3.6526
28     Topic5   60.515058            easy   91.974551   1.3234  -3.0868
14     Topic5   45.104429            code  178.204783   0.3680  -3.3807
79     Topic5    0.143005        security   32.310920  -3.6783  -9.1346
89     Topic5    0.141570           tools   32.564346  -3.6962  -9.1446
25     Topic5    0.144888            demo   33.633733  -3.7053  -9.1215
7      Topic5    0.149012    architecture   35.620087  -3.7346  -9.0934
29     Topic5    0.141855       extension   34.852711  -3.7621  -9.1426
76     Topic5    0.141871      repository   35.566585  -3.7822  -9.1425
44     Topic5    0.142023          kotlin   36.3412

In [68]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
wordnet_lemmatizer.lemmatize('dogs')

'dog'